# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120257e+02     1.630944e+00
 * time: 0.6009058952331543
     1     1.093380e+01     9.061809e-01
 * time: 1.6579689979553223
     2    -1.227509e+01     1.051355e+00
 * time: 2.1529269218444824
     3    -3.456244e+01     8.542085e-01
 * time: 2.915565013885498
     4    -4.833683e+01     5.669814e-01
 * time: 3.634446859359741
     5    -5.723304e+01     1.961135e-01
 * time: 4.354081869125366
     6    -5.996818e+01     9.536322e-02
 * time: 4.851265907287598
     7    -6.091622e+01     5.340587e-02
 * time: 5.356735944747925
     8    -6.132444e+01     6.637419e-02
 * time: 5.843098878860474
     9    -6.162853e+01     4.036974e-02
 * time: 6.328920841217041
    10    -6.183049e+01     2.926730e-02
 * time: 6.824652910232544
    11    -6.196628e+01     1.987717e-02
 * time: 7.326385021209717
    12    -6.203799e+01     1.751728e-02
 * time: 7.811548948287964
    13    -6.209274e+01     1.755538e-02
 * time: 8.296856880187988
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671076
    AtomicLocal         -18.8557750
    AtomicNonlocal      14.8522694
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485396 
    Xc                  -19.3336829

    total               -62.261666459050